#INVOICE PAYMENT DATE PREDICTION MODEL



# 1.1 Loading the given data set

In [ ]:
import pandas as pd
original_main_data=pd.read_csv("/content/drive/MyDrive/1805109.csv")
original_main_data.head()

FileNotFoundError: ignored

#1.2 Getting to know our Data

In [ ]:
# Finding the total number of rows or entries in the data set
len(original_main_data)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Getting first look at the data
original_main_data.info()

In [ ]:
# Checking for NULL values

original_main_data.isna().sum()

INFERENCE :

       1. The column " area_business " is completly empty

       2. The column " clear date " has some null values. These are intentionally null as these are the dates we have to predict.

       3. Most of the date columns are of type Object

# 2.1 Basic Preprocessing

In [ ]:
# Converting the Object type date columns to Date_Time objects


from datetime import datetime

original_main_data['posting_date']=pd.to_datetime(original_main_data['posting_date'])

original_main_data['document_create_date']=pd.to_datetime(original_main_data['document_create_date'],format='%Y%m%d')

original_main_data['document_create_date.1']=pd.to_datetime(original_main_data['document_create_date.1'],format='%Y%m%d')

original_main_data['due_in_date']=pd.to_datetime(original_main_data['due_in_date'],format='%Y%m%d')

original_main_data['baseline_create_date']=pd.to_datetime(original_main_data['baseline_create_date'],format='%Y%m%d')

original_main_data['due_in_date']=pd.to_datetime(original_main_data['due_in_date'])

original_main_data['clear_date']=pd.to_datetime(original_main_data['clear_date'])


In [ ]:
# Verifying our Conversions

original_main_data.info()

NOTE :

       1. The column "area_business" is completly NULL.It is not feasible for *NULL imputation* .

       2. Hence I ma dropping it as it will not give any useful information

In [ ]:
# Dropping column 'area_business'
original_main_data.drop('area_business',axis=1,inplace=True)

# 2.2 Checking for anamolies in our data

NOTE :

     1. From the given data dictionary we have,
       
        * document_create_date is the date on which the invoice document was created 

        * clear_date is the date when the invoice was cleared.

        Thus,it doesen't make entries to have clear_date before document_create_date

In [ ]:
#Checking for the above condition

len(original_main_data.loc[original_main_data['document_create_date.1'] > original_main_data['clear_date']]) 




Similarly, below I have checked for columns like 

               1. due_in_date

               2. posting_date

               3. baseline_create_date

  These columns should also not contain dates before document_create_date

In [ ]:
len(original_main_data.loc[original_main_data['document_create_date.1'] > original_main_data['due_in_date']])   

In [ ]:
len(original_main_data.loc[original_main_data['document_create_date.1'] > original_main_data['posting_date']])

In [ ]:
len(original_main_data.loc[original_main_data['document_create_date.1'] > original_main_data['baseline_create_date']])

Below is the percenatge of these rows with anomalies

In [ ]:
# number of abnormal data rows as as percentage

(134+2154)/len(original_main_data)

Now as these rows constiture of only about 4% of our data-set, It is better to drop them



In [ ]:
#Dropping the abnormal rows
original_main_data.drop(original_main_data[original_main_data['document_create_date.1'] > original_main_data['due_in_date']].index,inplace=True,axis=0)
original_main_data.drop(original_main_data[original_main_data['document_create_date.1'] > original_main_data['baseline_create_date']].index,inplace=True,axis=0)
len(original_main_data)

# 3.1 DATA SPLITTING

From observation ,my data is time based hence is to be split according to the given dates in ascending order.

1. The data is  splitted  into training and test sets based on a SPLIT_DATE.

2. The entries haveing data before this date belong to train set and after the split date belong to test set 

In [ ]:
#Splitting the data set into train and test sets

# The entries whicch do not have a clearing date are the ons we have to predict.

# Here I put those entries into test_set and the remaining into train set
training_data=original_main_data.loc[original_main_data['clear_date'].isna()==False,:]
test_data=original_main_data.loc[original_main_data['clear_date'].isna()==True,:]


In [ ]:
# Training Data
training_data.info()

In [ ]:
# Test Data
test_data.head()

In [ ]:
training_data.shape

In [ ]:
test_data.shape

Now testing for null values in training data

In [ ]:
training_data.isna().sum()

NOTE :
       1. There are still rows where the column "invoice_id" contains some NULL values

# 4.1 Exploratory Data Analysis (E.D.A)

In [ ]:
# Note the currency is of different types for different invoices
test_data['invoice_currency'].unique()

NOTE TO SELF :

By now Data has been split by date into training and testing sets (first 70-30%)split

In [ ]:
original_main_data[['clear_date','posting_date','document_create_date','due_in_date','baseline_create_date']].head()


 NOTE:-

     <#> Target value is the Column named 'due_in_date'

     <#> Bucket value can be calculated by the following :
              
              'aging_bucket'='clear_date' - 'due_in_date'
              

In [ ]:
training_data.groupby('invoice_currency').count()

Below we have our first look at the test_data.

*   The test data has 4188 entries all of which have "clear_date" as null

*   These are the entries we have to predict



In [ ]:
len(test_data)

In [ ]:
test_data.isna().sum()

In [ ]:
training_data.isna().sum()

The column "Invoice_id" is an id column .


*   Since it is not feasible to impute these values for the time being i am dropping it






In [ ]:
# Note invoice_id is missing 6 values, for time being I am dropping it.
training_data[pd.isnull(training_data['invoice_id'])==True]

In [ ]:
# these are the 6 empty rows in the training data-set
# Our test data contains no empty rows (confirmed)

train_data_new=training_data[pd.isnull(training_data['invoice_id'])!=True]
train_data_new

In [ ]:
train_data_new.isna().sum()

# Thus all null values are now finally removed

As seen earlier, the currency column has two values.

In other words they are not in the same unit.

In [ ]:
# Now solving the currency issue (Converting all CAD to USD throught)


train_data_new.groupby('invoice_currency').count()

In [ ]:
train_data_new.loc[train_data_new['invoice_currency']=='CAD'].head()

In [ ]:
# Conversion from CAD to USD 

# i CAD =0.79 CAD
import numpy as np
train_data_1=train_data_new
train_data_1['total_open_amount']=np.where(train_data_1.invoice_currency=='CAD',
                                             train_data_1.total_open_amount*0.79,
                                           train_data_1.total_open_amount)
train_data_1.tail()

In [ ]:
print("Data brfore conversion")
original_main_data.loc[45590,['invoice_currency','total_open_amount']]


In [ ]:
train_data=train_data_new
train_data.loc[45590,['invoice_currency','total_open_amount']]

As evident from above, all CAD have been converted tro USD and stored in train_data

In [ ]:
train_data.info()


In [ ]:
# Now we sort the training data based on invoice posting date

train_data.sort_values('document_create_date.1',inplace=True)
train_data.head()

# 4.2 CONSTANT COLUMNS

Taking a look at the "document_type" column

In [ ]:
train_data['document type'].unique()


In [ ]:
train_data.groupby("document type").count()

 This 'document type' column has all same values.

In [ ]:
train_data.groupby("isOpen").count()
# isOpen values are all same

The column "is_open" also conatains same values

# 4.3 DUPLICATE COLUMNS

In [ ]:
# Checking for duplicate columns
train_data.loc[train_data.invoice_id!=train_data.doc_id]

Hence we can infer that "invoice_id" and "doc_id" are duplicate columns

'invoce_id' and 'doc_id' are giving the same infomation 

So new info is derived from either of them 

So I can drop one of them.

In [ ]:
train_data.groupby("cust_payment_terms").count()  #esssential features

'isOpen' column is all 0 because this is test data and all invoives 
are unique.

This column also provides no new additional info.


In [ ]:
train_data['posting_id'].unique()

The column posting_id is also an constant column

In [ ]:
train_data.loc[train_data['document_create_date.1']!=train_data['baseline_create_date'],:]

These two columns are different namely document_create_date.1 and baseline_create_date

 <#> Target value is the Column named 'due_in_date'
 
 <#> Bucket value can be calculated by the following :
 
          'aging_bucket'='clear_date' - 'due_in_date

In [ ]:
train_data['clear_date']=pd.to_datetime(train_data['clear_date'])
train_data.info()

In [ ]:
train_data['due_in_date']=pd.to_datetime(train_data['due_in_date'])
#train_data['due_in_date']=train_data['due_in_date'].dt.date
train_data['delay']=(train_data['clear_date']-train_data['due_in_date'])

train_data['delay']=(train_data['delay']/ np.timedelta64(1, 'D')).astype(int)

train_data.tail()

# Splitting into Train , Validation_1 and Validation_2

Now I am spliting the train data into train : Val 1 :Val 2 = 7:15:15


In [ ]:
len(train_data)

In [ ]:
final_train_data,val=np.split(train_data,[int(.7*len(train_data))])
print("Length of final_train_data is ",len(final_train_data),
      "\n Length of val is ",len(val))

In [ ]:
val1,val2=np.split(val,[int(.5*len(val))])
print("Length of val1 is ",len(val1),
      "\nLength of val2 is ",len(val2))

In [ ]:
# Check for missing rows

30556+6548*2


Hence the data is effectively split witout missing any value

In [ ]:
# Creating Dictionary which will later be used for mapping
# This is to be done before splitting into x and y
BC_DICT=final_train_data.groupby('business_code')['delay'].mean().to_dict()
BC_DICT

Thus , now the split into train_final,validation-1 and valdation_2 has been done

Now Splitting the data into X_train and y_train

In [ ]:
X_train = final_train_data.copy()
X_train.drop('delay', axis=1, inplace=True)
y_train = final_train_data['delay']
final_train_data.shape

In [ ]:
final_train_data.isna().sum()

In [ ]:
X_train.shape

In [ ]:
y_train.shape

Thus,the data is split into X_train and y_train

In [ ]:
y_train.head()

# 5.1 Data Visualization and Pattern Analysis

Testing for patterns in data

Analysis of due_in_date with delay

In [ ]:

days=X_train['due_in_date'].dt.day

# Define Day maping function
def day_to_bin(day):
  if day<=7:
    return 1
  elif day>7 and day<=14:
    return 2
  elif day>14 and day<=21:
    return 3
  else:
    return 4

day_bins=np.array(days.map(day_to_bin))
np.unique(day_bins)

Effect of Months on delay:

In [ ]:
#Trying monthsas feature
due_months=X_train['due_in_date'].dt.month
import matplotlib.pyplot as plt
fig,ax=plt.subplots(figsize=(7,3))
plt.xlabel('months')
ax.hist(due_months,bins=[0,1,2,3,4,5,6,7,8,9,10,11,12])
plt.show()

From here It can be infered that the month in which the client makes payment may be a useful feature as no two months have the same no of due clearence.

Effect of Amount on delay

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(9,5))
plt.xlabel('total_open_amount')
plt.ylabel('delay')
x=(final_train_data['total_open_amount'])
y=final_train_data['delay']
plt.scatter(x,y,c="blue")
plt.show()

INFERENCE:

Clients with less open_amount tebd to delay more than the ines with hogher total_open_amount

In [ ]:
train_data.tail()

In [ ]:
train_data.info()

Examining the distribution of total_open_amount

In [ ]:
import seaborn as sns
sns.distplot(a=X_train['total_open_amount'], kde=False)

The distribution is right skewed as seen from graph

I will later use log transform to make it as close to a normal distribution as follows

Range of Delay

In [ ]:
# RAnge of Delay
range=np.ptp(train_data['delay'])
range

In [ ]:
train_data['delay'].std()

In [ ]:
train_data['delay'].mean()/365


Effect of business year on Delay

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x=train_data.loc[:100,'buisness_year']
y=train_data.loc[:100,'delay']
fig, ax = plt.subplots(figsize=(9,6))
plt.xlabel('buisness_year')
plt.ylabel('delay')
plt.scatter(x,y)

In [ ]:
train_data.groupby('buisness_year').sum()

In [ ]:
# Scaling total_open_amount by a factor of 1000
train_data['total_amount_by_1000']=train_data['total_open_amount']/1000
train_data.tail()

A Plot between cust_number and total_amount

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(train_data.loc[:,'cust_number'],train_data.loc[:,'total_amount_by_1000'])
plt.xlabel('cust_number')
plt.ylabel('total_amount')
plt.show()

Effect of Customer Payment Terms on Delay

In [ ]:
# Types of terms in data
terms=train_data.cust_payment_terms.unique()
terms

In [ ]:
# Visual Plot
df=pd.DataFrame(train_data.groupby('cust_payment_terms')['total_amount_by_1000'].mean()).reset_index()
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(9,6))
plt.xlabel('cust_payment_terms')
plt.ylabel('total_amount(scaled)')
ax.plot(df['cust_payment_terms'],df['total_amount_by_1000'])
fig.show()



Information given by this is not sufficient so moving on to bar-plot

In [ ]:
# Plotting bar-plot using seaborn
sns.set(style="whitegrid")
fig=plt.figure(figsize=(30,20))
terms=train_data.groupby('cust_payment_terms')['delay'].sum()
sns.barplot(y=terms.index,x=terms)

The above plot shows customer_payment_terms is an essential factor for delay

Hence confirmed that customer payment terms is an Nominal Column

# 6.1 Encoding

Now doing One Hot encoding for Customer Payment Terms

Sadly One hot encoding did not work , It created 68 columns

Now moving On to Target Encoding

In [ ]:
# Creating a new column that is Invoice Currency code range(0,1)

X_train['inv_curr_code']=np.where(X_train.invoice_currency=='CAD',0,1)
X_train.loc[X_train.invoice_currency=='CAD',:]

# Now we can drop the column invoice_currency column
X_train.drop('invoice_currency',axis=1,inplace=True)
X_train.head()


Now encoding the column 'cust_payment_terms' using Target Encoding

In [ ]:
len(X_train.cust_payment_terms.unique())

# These are the number of unique Labels in the cust_payment_terms column
# These are to be encoded


In [ ]:
X_train.groupby('cust_payment_terms').count()

NOTE:



*   As from our graph we nmotice that the customer payment terms gighly affect the delay.
*   Thus for this I created a new variable "avg_delay_as_per_terms"
*   It is defined as the median of all the customer who agreed  on the same terms






In [ ]:
# TARGET ENCODING

X_train['target'] = y_train # adding it again for demonstration of target encoding

target_mapper = X_train.groupby('cust_payment_terms')['target'].median().to_dict()
print(target_mapper)

X_train.drop('target',axis=1,inplace=True) # Removing it back

X_train['avg_delay_as_per_terms'] = X_train['cust_payment_terms'].map(target_mapper)
X_train.head()

Hence , now the target encoding of the column 'cust_payment_terms' has been done

The new column which is supposed to replace ot is named avg_delay_as_per_terms

Now encoding Customer_name column

Now Encoding Customer Name

In [ ]:
# Unique labels for customer name
len(X_train.name_customer.unique())

In [ ]:
# Target Encoding for customer_name
temp=X_train.copy()
temp['delay']=y_train
col_map=temp.groupby('name_customer')['delay'].mean().to_dict()
col_map


Now merging this with the X_train

In [ ]:
'''
new=X_train.merge(col_map,how='left',left_on=['name_customer'],right_on=['name_customer'])
new.loc[45:50,:]
'''
X_train['cust_name(ENC)']=X_train['name_customer'].map(col_map)
X_train.head()

In [ ]:
X_train.head()

Thus by now customer_names have been encoded as "cust_name(ENC)"

Business_year column is already in float and requires no changes.

This  column is also essential as delay depends on business year as per graph

In [ ]:
X_train.groupby('buisness_year').size()

In [ ]:
len(X_train.business_code.unique())

Encoding Customer Number

In [ ]:
# Unique customer numbers
len(X_train.cust_number.unique())

In [ ]:
# Target Encoding for cust_number
temp=X_train.copy()
temp['delay']=y_train
MAPPER=temp.groupby('cust_number')['delay'].mean().to_dict()
MAPPER

In [ ]:
# mapping cust_number(ENC)
X_train['cust_num(ENC)']=X_train['cust_number'].map(MAPPER)
X_train.head()

Now ,
       The column cust_number has  beeen encoded as 'cust_num(ENC)'

# 7.1 Feature Engineering

Here I am creating a new feature called as "time_given"
*   This feature denotes the time given to a particular client to clear his payment dues
*   time_given=due_in_date - document_create_date.1





In [ ]:
# Creating new feature
X_train['time_given']=X_train['due_in_date']-X_train['document_create_date.1']
X_train.head()

The time difference is of type "TimeDelta"

Now converting it to int type

In [ ]:
# Conversion from TimeDelta to int
X_train['time_given']=(X_train['time_given']/ np.timedelta64(1, 'D')).astype(int)
X_train.info()

Let us take a look at the scatter plot and visualize how it affects delay

In [ ]:
# Sctter plot of 
plt.scatter(y_train,X_train['time_given'],color='black')
plt.xlabel("time_given")
plt.ylabel("delay")
plt.title("delay Vs. time_given")
plt.show()

INFERENCE:

*   It is observed from the graph that if more time is given to a client , delay is less and vice-versa




The doc_id column has many unique values as  seen from below

In [ ]:
# analysis of unique values in doc_id
len(X_train.doc_id.unique())

As stated earlier now we engineeer a new column to replace "total_open_amount" by using log transform

In [ ]:
#Normalizing (log2 Transformation)
X_train['total_open_amount_1']=np.log2(X_train['total_open_amount'])

In [ ]:
import seaborn as sns
sns.distplot(a=X_train['total_open_amount_1'], kde=False)

This plot seems closer to  a normal distribution as compared to before

In [ ]:
# Target encoding Business Code
def BC_MAPPER(key):
  if key not in BC_DICT.keys():
    return 2 #default value
  else:
    return BC_DICT[key]

Now below we create two new columns namely,


*   business_code(ENC) --> encoded form of business_code
*   due_month -->month of due_in_date



In [ ]:
# Calling Encoder
X_train['business_code(ENC)']=X_train['business_code'].map(BC_MAPPER)
# Creating new feature due_month
X_train['due_month']=X_train['due_in_date'].dt.month

As business code is now in float we check its impact on delay

In [ ]:
# Impact of business code on delay
import matplotlib.pyplot as plt
plt.xlabel('business code')
plt.ylabel('delay')
plt.scatter(X_train['business_code(ENC)'],y_train,color='black')
plt.show()

NOTE

From graph it is sceen that business codes encoded as higher integers tend to show less delay

now business code has been target encoded

In [ ]:
# Creating feature due_day
X_train['due_day']=X_train['due_in_date'].dt.day
# Document create month
X_train['doc_month']=X_train['document_create_date.1'].dt.month
# Document create day
X_train['doc_day']=X_train['document_create_date.1'].dt.day

# 8.1 Feature Selection

Filter Methods

In [ ]:
# Using Filter methods 

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold

Using Variance threshold :

**For** using Variance Threshold all features should be numerical which is not currently the case as seen below

In [ ]:
X_train.info()

So I copy X_train into a new df and drop all non-numeric columns so it can be passed to VAriance Threshold 

In [ ]:
train=X_train.copy()
train.drop(['business_code', 'cust_number','name_customer','clear_date','posting_date','document_create_date', 'document_create_date.1', 'due_in_date' ,'document type','posting_id','baseline_create_date',	'cust_payment_terms','isOpen'],axis=1,inplace=True)
train.drop(['doc_id','invoice_id','total_open_amount'],axis=1,inplace=True)
train.info()  

In [ ]:
#Passing a value of zero for the parameter will filter all the features with zero variance
constant_filter = VarianceThreshold(threshold=0)
#we need to simply apply this filter to our training set as shown in the following example
constant_filter.fit(train)
#the number of non-constant features.
len(train.columns[constant_filter.get_support()])

Thus we can say that all the 12 columns are not constant in nature.

This is also verified from below

In [ ]:
#Checking for constant columns
constant_columns = [column for column in train.columns
                    if column not in train.columns[constant_filter.get_support()]]

print(len(constant_columns))

Effect of invoice_currency code on delay

In [ ]:
# Plot
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="ticks", color_codes=True)
train=X_train.copy()
train['delay']=y_train
sns.catplot(x="inv_curr_code", y="delay",kind="bar", data=train)  
sns.catplot(x="buisness_year",y="delay",jitter=False,data=train)

INFERENCE--> Clients who use CAD as invoice_currency tend to delay more than those who use USD


# 8.2 Corelation and Covariace

Covariance & Corelation using HeatMap

In [ ]:
#visualisation
train=X_train.copy()
train.drop(['business_code', 'cust_number','name_customer','clear_date','posting_date','document_create_date', 'document_create_date.1', 'due_in_date' ,'document type','posting_id','baseline_create_date',	'cust_payment_terms','isOpen'],axis=1,inplace=True)
corr=train.corr()
plt.figure(figsize=(14,8))
sns.heatmap(corr,annot=True)

This shows the covariance between different columns of the X_train data.

*   The co-relation betweeen target and other columns should be high.
*   The co-relation among other columns should be asa low as possible.



# 9.1 Fitting into Model

Now finally fitting the model

In [ ]:
X_train.shape

In [ ]:
#Saving X_train to another variable (BACKUP)
just_in_case=X_train.copy()

In [ ]:
X_train.drop(['business_code', 'cust_number','name_customer','clear_date','posting_date','document_create_date', 'document_create_date.1', 'due_in_date' ,'document type','posting_id','baseline_create_date',	'cust_payment_terms','isOpen'],axis=1,inplace=True)
X_train.info()

In [ ]:
X_train.drop(['doc_id','invoice_id','total_open_amount'],axis=1,inplace=True)

In [ ]:
X_train.shape

# 9.2 Mapper Function

Now , time has come to make our pre-processing function to make out test data ready,
for making predictions.

In [ ]:
# Defining Mapper Functions

def cpt_mapper(key):
  '''
  maps customer payment terms
  '''
  if key not in target_mapper.keys():
    return X_train['avg_delay_as_per_terms'].mean()
  else:
    return target_mapper[key]

def cn_mapper(key):
  '''
  maps customer name
  '''
  if key not in col_map.keys():
    return X_train['cust_name(ENC)'].mean()
  else:
    return col_map[key]

def cno_mapper(key):
  '''
  maps customer number
  '''
  if key not in MAPPER.keys():
    return X_train['cust_num(ENC)'].mean()
  else:
    return MAPPER[key]

def BC_MAPPER(key):
  '''
  maps business code
  '''
  if key not in BC_DICT.keys():
    return 2 #default value
  else:
    return BC_DICT[key]


# 9.3 Preprocessing Function

In [ ]:
def preprocess(X_Set,y_Set):
  '''
  This function takes a data-set and preprocess it 
  exactly the same as X-train data
  '''
  # Creating a new column that is Invoice Currency code {0,1}

  X_Set['inv_curr_code']=np.where(X_Set.invoice_currency=='CAD',0,1)
 
  # Now we can drop the column invoice_currency column
  X_Set.drop('invoice_currency',axis=1,inplace=True)
  
  # TARGET ENCODING
  
  # Customer payment terms


  X_Set['avg_delay_as_per_terms']= X_Set['cust_payment_terms'].map(cpt_mapper)

  # Customer Name

  X_Set['cust_name(ENC)']=X_Set['name_customer'].map(cn_mapper)
  
  # Customer Number
  X_Set['cust_num(ENC)']=X_Set['cust_number'].map(cno_mapper)

  # Creating New column " time_given "

  X_Set['time_given']=X_Set['due_in_date']-X_Set['document_create_date.1']
  X_Set['time_given']=(X_Set['time_given']/ np.timedelta64(1, 'D')).astype(int)

  # Log transforming total_open_amount
  X_Set['total_open_amount_1']=np.log2(X_Set['total_open_amount'])

  # Mapping business code as business_code(ENC)
  X_Set['business_code(ENC)']=X_Set['business_code'].map(BC_MAPPER)

  #Creting new feature due month
  X_Set['due_month']=X_Set['due_in_date'].dt.month

  # Creating feature due_day
  X_Set['due_day']=X_Set['due_in_date'].dt.day
  # Document create month
  X_Set['doc_month']=X_Set['document_create_date.1'].dt.month
  # Document create day
  X_Set['doc_day']=X_Set['document_create_date.1'].dt.day

  #Dropping Columns
  X_Set.drop(['business_code', 'cust_number','name_customer','clear_date','posting_date','document_create_date', 'document_create_date.1', 'due_in_date' ,'document type','posting_id','baseline_create_date',	'cust_payment_terms','isOpen'],axis=1,inplace=True)
  X_Set.drop(['doc_id','invoice_id','total_open_amount'],axis=1,inplace=True)
  
  return X_Set,y_Set

# 10.1 Validation

Spliting validation sets into x and y

In [ ]:
# Validation 1
X_Val1=val1.copy()
y_Val1=X_Val1['delay']
X_Val1.drop('delay',axis=1,inplace=True)

In [ ]:
#Validation 2
X_Val2=val2.copy()
y_Val2=X_Val2['delay']
X_Val2.drop('delay',axis=1,inplace=True)

# 10.2 Fitting the model

In [ ]:
#Fitting the model
from sklearn.ensemble import RandomForestRegressor

clf=RandomForestRegressor()
clf.fit(X_train,y_train)

Predicting on Validation sets

In [ ]:
# Predicting values for Validation Sets
x_val1,y_val1=preprocess(X_Val1,y_Val1)
x_val2,y_val2=preprocess(X_Val2,y_Val2)
val1_preds=clf.predict(x_val1)
val2_preds=clf.predict(x_val2)

In [ ]:
# Creating a Evluation Function
from sklearn.metrics import r2_score,mean_squared_error
def evaluate_model(labels,predictions):
  print("RMSE : ",mean_squared_error(labels,predictions,squared=False))
  print("R^2 SCORE : ",r2_score(labels,predictions))


In [ ]:
#Evluating the RandomForest Refressor
evaluate_model(y_val1,val1_preds)
evaluate_model(y_val2,val2_preds)


In [ ]:

# Trying Lssso Reggressor
from sklearn import linear_model
reg = linear_model.Lasso(alpha=0.9)
reg.fit(X_train,y_train)
evaluate_model(y_val1,reg.predict(x_val1))
evaluate_model(y_val2,reg.predict(x_val2))


In [ ]:

#Trying RidgeRegression Model
from sklearn import linear_model
ridge = linear_model.Ridge(alpha=.1)
ridge.fit(X_train,y_train)
val1_preds=ridge.predict(x_val1)
val2_preds=ridge.predict(x_val2)
evaluate_model(y_val1,ridge.predict(x_val1))
evaluate_model(y_val2,ridge.predict(x_val2))


From my experiments I found that the random forest regressor gave the best base results for me



*   Below is the list of all other models I tried .
*   I found that the best result was given by RAndomForestRegressor()



In [ ]:

#Trying SVC kernel='linear'
from sklearn import svm
regr = svm.SVR()
regr.fit(X_train, y_train)


In [ ]:
y2_preds=ridge.predict(x_val1)
mean_squared_error(y_val2,y2_preds,squared=False)


In [ ]:

#Trying out GradientBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
est = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,
    max_depth=1, random_state=0, loss='ls').fit(X_train, y_train)
est.predict(x)
y2_preds=ridge.predict(x)
mean_squared_error(y2,y2_preds,squared=False)
r2_score(y2,y_preds)



In [ ]:
'''
#Lasso Variant
from sklearn import linear_model
reg = linear_model.LassoLars(alpha=.1)
reg.fit(X_train,y_train)
mean_squared_error(y2,y2_preds,squared=False)
'''

# 10.3 HyperParameter Tuning

Fitting model with n_estimators=100

In [ ]:
# Fitting 
clf=RandomForestRegressor(n_estimators=100)
clf.fit(X_train,y_train)

In [ ]:
#Now making predictions on the same validation sets
val1_pred=clf.predict(x_val1)
val2_pred=clf.predict(x_val2)


In [ ]:
# Testing the accuracy
evaluate_model(y_val1,val1_preds)
evaluate_model(y_val2,val2_preds)

As we can see tuning is not giving better results so I have used the base model

# 11.1 Making predictions on the test data

Now I am making predictions on the test data.

In [ ]:
df=test_data.copy()

It is to be noted that the test data is not yet processed and contains NULL values in the clear date column

We need to fill these values with our predicted dates and finally bin them according to the following buckets.

● 0-15 days

● 16-30 days

● 31-45 days

● 46-60 days

● Greater than 60 days




Now this is ready for preprocessing function

In [ ]:
# X_test and y_test creation
y_test=[]
X_test,y_test=preprocess(test_data,y_test)
X_test.info()

Making Predictions Using our RandomForest Regressor model

In [ ]:
y_test=clf.predict(X_test)
y_test

Rounding float to int

AS we are predicting the number of days of delay ,we require integers so here I round up the floating points to int

In [ ]:
y_test_rounded=np.round(y_test).astype(int)
y_test_rounded

In [ ]:
# Adding y_test to df
df['delay']=y_test_rounded
df.head()

In [ ]:
# Converting due_in_date to Date-time
df['due_in_date']=pd.to_datetime(df['due_in_date'],format='%Y%m%d')
# COnverting delay to time delta
df['delay']=pd.to_timedelta(df['delay'], unit='D')
# Creating new column clear_date
df['clear_date']=df['due_in_date']+df['delay']
df.head()

Thus we can predict the clear date of an invoice

In [ ]:
# Function for bucketing
from datetime import datetime, timedelta
def bucketizer(delay):
  '''
  This function is used to map the delays to their 
  respective bucket
  '''
  delay=delay.total_seconds() / timedelta (days=1).total_seconds()
  if delay <=15 :
    return "0-15 days"
  elif delay >=16 and delay <=30:
    return "15-30 days"
  elif delay >=31 and delay <=45:
    return "31-45 days"
  elif delay>=46 and delay<=60:
    return "46-60 days"
  else:
    return "greater than 60 days"

In [ ]:
# Now Creating the Bucket column using this function
df['bucket']=df['delay'].map(bucketizer)
df.head()

In [ ]:
df.groupby('bucket').count()

Hence we have  bucketized all the test data entries and achieved our goal.

It is furthet clear from below,

In [ ]:
df[['due_in_date','delay','clear_date','bucket']].head()

That's all !!! Thank You for ur time :)